In [33]:
%matplotlib inline
from datetime import datetime, date, timedelta  # working with date and time
import matplotlib.pyplot as plt      # graphing
import numpy as np                   # math
import os                            # useful for handling filenames etc.
import pandas as pd                  # manipulating data
from scipy.stats import pearsonr     # calculates the Pearson correlation coefficient and p-value
import seaborn as sns                # makes matplotlib beautiful
sns.set_style('darkgrid')
import weather_station_utils as wsu  # helper functions for WeatherUnderground data

import matplotlib as mpl             # control formatting
mpl.rcParams['axes.titlesize'] = 16
mpl.rcParams['axes.titleweight'] = 'semibold'

# interactive graphs
import bokeh
from bokeh.io import output_notebook, show, push_notebook
from bokeh.plotting import figure
from bokeh.layouts import row, column
from bokeh.models import DatetimeTickFormatter, HoverTool
from bokeh.palettes import Category10_5
output_notebook()

Loading BokehJS ...

# Longer-term Patterns in Data from Pinewood High School
Pinewood High School has had the new array of sensors (including weather sensors as well as a D3S) for the longest time out of any school&mdash;they have had one from February. To begin, we will look for a phenomenon called radon washout. When it rains, the rain washes down radon gas, bringing it closer to the ground. This should be recorded as a spike in radiation levels. So we let's see if we see such a pattern in the data collected from Pinewood.

We have already binned the data. It can be found in the [`binned_data_long_term`](binned_data_long_term) directory. Let's get data from [WeatherUnderground](https://www.wunderground.com/personal-weather-station/dashboard?ID=KCASANTA463) and bin it.

### Load the Binned Data

In [14]:
LOCATION_ID = 'KCASANTA463'
LOC_PINEWOOD = 'pine'
INTERVAL = 60 * 60 * 12  # half a day
DATA_DIR = 'binned_data_long_term'

pinewood_data = wsu.get_all_binned(interval=INTERVAL, data_dir=DATA_DIR, location=LOC_PINEWOOD)
pinewood_data

,unix_time,co2,radiation,pgradiation,humidity,temperature,pressure,pm1,pm25,pm10
0,1447986433,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1448029633,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1448072833,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1448116033,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1448159233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1448202433,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1448245633,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1448288833,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1448332033,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1448375233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The `NaN`s are there because we started binning from way before the sensors began making measurements. We have complete data (over the current binning interval of 14 days) from the time that the sensors started measuring data. We can conveniently drop all the `NaN`s using the `DataFrame.dropna` function.

In [3]:
pinewood_data.dropna(how='any', axis=0, inplace=True)
pinewood_data

,unix_time,co2,radiation,pgradiation,humidity,temperature,pressure,pm1,pm25,pm10
57,1516933633,451.130013,1177.750912,1.773544,60.877930,15.938773,1014.535017,0.793091,1.514119,1.916546
58,1518143233,463.093007,1166.862237,1.759135,59.361704,10.995078,1014.071926,0.387161,0.775337,1.039358
59,1519352833,482.741898,1150.011801,1.731412,67.324373,10.449438,1015.486184,0.026882,0.134714,0.200881
60,1520562433,390.867671,1150.014859,1.913707,75.127967,14.887097,1010.257778,1.673688,2.515052,3.144380
61,1521772033,462.940326,1175.614202,2.125255,58.580959,16.398065,1013.414128,0.530487,1.062072,1.418988
62,1522981633,481.858286,1154.364916,2.111242,65.623733,15.223094,1014.603076,0.144520,0.440790,0.661948
63,1524191233,471.693303,1179.484255,2.147514,60.606181,17.435269,1012.652992,2.334551,4.051190,5.392062
64,1525400833,465.879308,1175.532095,2.139807,55.198832,19.318977,1010.937137,1.051360,2.021609,2.639564
65,1526610433,459.462586,1185.017119,2.289959,62.031001,18.314981,1009.832145,2.274007,4.030904,5.577893
66,1527820033,475.905820,1193.305473,2.312088,52.438744,21.557348,1008.313233,1.394192,2.640367,3.597384


Yay! No more `NaN`s. Now let's get data from WeatherUnderground.

### Get Data from WeatherUnderground

In [9]:
start = date.fromtimestamp(pinewood_data.iloc[0]['unix_time']) + timedelta(days=5)
end = date.fromtimestamp(pinewood_data.iloc[-1]['unix_time']) + timedelta(days=17)

print(f'Getting data from {start} to {end}, from {LOCATION_ID}.')
pinewood_ws = wsu.get_ws_data_by_time(start, end, LOCATION_ID)

# first, save the DataFrame as a `.csv` file
# before saving, we need to rename the column `Time` to `deviceTime_unix`
header = pinewood_ws.columns.values.tolist()
print(header)

Getting data from 2018-01-30 to 2018-07-15, from KCASANTA463.
['Dewpoint', 'HourlyPrecipIn', 'Humidity', 'Pressure', 'SolarRadiationWatts/m^2', 'Temperature', 'Time', 'WindDirection', 'WindDirectionDegrees', 'WindSpeedGustMPH', 'WindSpeedMPH', 'dailyrainin']


Column index 6 is called `Time`, so we need to change that to `deviceTime_unix` before binning, because [`time_binning.py`](time_binning.py) looks for that column in the DataFrame provided.

In [6]:
header[6] = 'deviceTime_unix'
print(header)

# comment out this line because we don't want to save the `.csv` file every time we rerun the notebook
pinewood_ws.to_csv(f'wunderground_data/data_{LOC_PINEWOOD}_long_term.csv',
                   na_rep='nan', index=False, header=header)

pinewood_ws

['Dewpoint', 'HourlyPrecipIn', 'Humidity', 'Pressure', 'SolarRadiationWatts/m^2', 'Temperature', 'deviceTime_unix', 'WindDirection', 'WindDirectionDegrees', 'WindSpeedGustMPH', 'WindSpeedMPH', 'dailyrainin']


,Dewpoint,HourlyPrecipIn,Humidity,Pressure,SolarRadiationWatts/m^2,Temperature,Time,WindDirection,WindDirectionDegrees,WindSpeedGustMPH,WindSpeedMPH,dailyrainin
0,7.111111,0.0,85.0,1013.880765,0.0,9.500000,1516867320,ESE,105.0,9.8,6.5,0.0
1,7.611111,0.0,87.0,1013.880765,0.0,9.611111,1516867680,ESE,114.0,7.4,6.5,0.0
2,7.611111,0.0,88.0,1013.542127,0.0,9.500000,1516867980,ESE,111.0,7.4,5.6,0.0
3,7.500000,0.0,88.0,1013.880765,0.0,9.388889,1516868280,SE,139.0,7.4,5.6,0.0
4,7.388889,0.0,88.0,1013.880765,0.0,9.277778,1516868580,SE,139.0,7.4,5.1,0.0
5,7.222222,0.0,86.0,1013.880765,0.0,9.388889,1516868880,ESE,104.0,4.9,4.9,0.0
6,6.500000,0.0,82.0,1014.219402,0.0,9.388889,1516869180,ESE,119.0,4.9,3.6,0.0
7,5.277778,0.0,75.0,1013.880765,0.0,9.500000,1516869480,SE,138.0,4.9,2.9,0.0
8,5.111111,0.0,73.0,1013.880765,0.0,9.722222,1516869780,SE,139.0,4.9,3.6,0.0
9,5.611111,0.0,75.0,1013.880765,0.0,9.777778,1516870080,SE,139.0,4.9,3.4,0.0


Now we will bin the data by appropriately modifying [`multi_bin.py`](multi_bin.py) and running it using the command
```
python multi_bin.py pine
```

Next, we will load the binned weather station data.

In [15]:
winddir_data = pd.read_csv(f'{DATA_DIR}/ws_{LOC_PINEWOOD}_data_WindDirectionDegrees_{INTERVAL}.csv',
                            header=0, names=['unix_time', 'WindDirection'], usecols=[1])
windspd_data = pd.read_csv(f'{DATA_DIR}/ws_{LOC_PINEWOOD}_data_WindSpeedGustMPH_{INTERVAL}.csv',
                            header=0, names=['unix_time', 'WindSpeedGustMPH'], usecols=[1])
temp_data = pd.read_csv(f'{DATA_DIR}/ws_{LOC_PINEWOOD}_data_Temperature_{INTERVAL}.csv',
                            header=0, names=['unix_time', 'Temperature'], usecols=[1])
pressure_data = pd.read_csv(f'{DATA_DIR}/ws_{LOC_PINEWOOD}_data_Pressure_{INTERVAL}.csv',
                            header=0, names=['unix_time', 'Pressure'], usecols=[1])
humidity_data = pd.read_csv(f'{DATA_DIR}/ws_{LOC_PINEWOOD}_data_Humidity_{INTERVAL}.csv',
                            header=0, names=['unix_time', 'Humidity'], usecols=[1])
rainfall_data = pd.read_csv(f'{DATA_DIR}/ws_{LOC_PINEWOOD}_data_dailyrainin_{INTERVAL}.csv',
                            header=0, names=['unix_time', 'Rainfall'], usecols=[1])

pinewood_dosenet_and_ws = pd.concat([pinewood_data, winddir_data, windspd_data,
                                     temp_data, pressure_data, humidity_data, rainfall_data], axis=1)
pinewood_dosenet_and_ws.dropna(axis=0, how='any', inplace=True)
pinewood_dosenet_and_ws['unix_time'] = pinewood_dosenet_and_ws['unix_time'].astype(int)
pinewood_dosenet_and_ws

,unix_time,co2,radiation,pgradiation,humidity,temperature,pressure,pm1,pm25,pm10,WindDirection,WindSpeedGustMPH,Temperature,Pressure,Humidity,Rainfall
1609,1517495233,560.682258,1171.290323,1.791011,37.879355,22.444677,1013.192419,0.000000,0.012131,0.014754,211.098592,5.585915,23.381455,1012.910163,14.035211,0.0
1610,1517538433,553.922778,1190.327027,1.855140,77.146000,11.040276,1014.894828,0.197067,0.663067,0.879200,239.000000,0.208333,21.509259,1013.711446,19.250000,0.0
1611,1517581633,526.841905,1157.600000,1.697321,52.678041,19.437095,1015.379324,0.168411,0.206424,0.226225,297.833333,2.463333,22.622222,1014.862814,18.133333,0.0
1612,1517624833,476.004966,1170.394805,1.722222,75.834762,12.044694,1015.269728,0.054490,0.421565,0.558231,86.340426,5.700709,18.689913,1015.336186,19.602837,0.0
1613,1517668033,463.680625,1162.998667,1.700457,48.668056,20.706042,1015.078750,0.212517,0.496463,0.657347,224.119718,5.488732,24.183881,1015.528642,13.971831,0.0
1614,1517711233,491.586181,1172.348684,1.728829,74.921319,12.895972,1013.816181,0.078792,0.789396,0.942148,116.226950,3.487943,18.635934,1014.517211,18.929078,0.0
1615,1517754433,368.907514,1171.613423,1.859072,54.300629,19.902937,1013.715524,0.013265,0.469184,0.627823,237.770833,5.004861,23.483025,1014.348743,14.263889,0.0
1616,1517797633,248.637439,1199.984828,1.847222,79.338276,11.472690,1012.760483,1.178784,2.725068,3.546351,115.771429,2.725714,16.427381,1013.053522,18.992857,0.0
1617,1517840833,494.420274,1179.355556,1.725000,53.010139,20.492708,1011.033750,0.877724,2.019586,2.654276,217.359155,6.335211,22.615023,1011.205051,14.633803,0.0
1618,1517884033,472.882708,1185.833566,1.727778,75.241736,12.150833,1011.199861,0.343333,1.111875,1.681389,248.971831,1.579577,18.211268,1010.616013,19.007042,0.0


In [16]:
pinewood_dosenet_and_ws['Rainfall'].value_counts()

0.000000    198
0.010000      6
0.004085      1
0.004684      1
0.001560      1
0.018794      1
0.063546      1
0.082695      1
0.003546      1
0.067801      1
0.004583      1
0.006620      1
0.033285      1
0.220000      1
0.019433      1
0.004648      1
0.071786      1
0.029650      1
0.003776      1
0.004088      1
0.041500      1
0.003826      1
0.003803      1
Name: Rainfall, dtype: int64

As we can see, most of the time, we got no rain. But we do have some days with more rain than others. Let's see how that compares to any spikes we might see in the radiation data.

### Visualizing Radon Washout
We will use the [`Bokeh`](https://bokeh.pydata.org/en/latest/) library for this first.

In [31]:
pinewood_dosenet_and_ws['datetime'] = pd.to_datetime(pinewood_dosenet_and_ws['unix_time'], unit='s')
# we need to normalize the radiation and rainfall columns before we can meaningfully visulaize them
# otherwise we will see this:
p = figure(width=600, height=400, x_axis_type='datetime')
p.multi_line(xs=[pinewood_dosenet_and_ws['datetime'] for _ in range(2)],
             ys=[pinewood_dosenet_and_ws['Rainfall'],
                 pinewood_dosenet_and_ws['radiation']])
show(p)

So we will now normalize the `radiation` and `Rainfall` columns.

In [38]:
pinewood_dosenet_and_ws['norm_rad'] = pinewood_dosenet_and_ws['radiation'] / max(pinewood_dosenet_and_ws['radiation'])
pinewood_dosenet_and_ws['norm_rain'] = pinewood_dosenet_and_ws['Rainfall'] / max(pinewood_dosenet_and_ws['Rainfall'])

palette = Category10_5
p = figure(width=600, height=400, x_axis_type='datetime', title='Rainfall vs. Radiation')
# p.multi_line(xs=[pinewood_dosenet_and_ws['datetime'] for _ in range(2)],
#              ys=[pinewood_dosenet_and_ws['norm_rain'],
#                  pinewood_dosenet_and_ws['norm_rad']],
#              line_color=palette[:2])
p.line(x=pinewood_dosenet_and_ws['datetime'],
       y=pinewood_dosenet_and_ws['norm_rain'],
       legend='Rainfall', color='blue')
p.line(x=pinewood_dosenet_and_ws['datetime'],
       y=pinewood_dosenet_and_ws['norm_rad'],
       legend='Radiation', color='orange')
p.add_tools(HoverTool(tooltips=[('Time', '$x'), ('Value', '$y')]))
p.xaxis.axis_label = 'Time'
p.yaxis.axis_label = 'Normalized Rainfall and Radiation'
show(p)

The peaks in the two series line up pretty well, especially the larger peaks. And the small delay between the rainfall and radiation spikes can be explained as follows: as rain falls, it washes down radon gas, which accumulates slowly, and reaches peak concentration some time after maximum rainfall. Let's look at the value of Pearson's correlation coefficient for these two series and examine the significance of the correlation coefficient obtained.

In [40]:
r, p = pearsonr(pinewood_dosenet_and_ws['norm_rad'], pinewood_dosenet_and_ws['norm_rain'])
print(f"Pearson's correlation coefficient is {r:.3f}, and the two-tailed p-value is {p:.3f}.")

Pearson's correlation coefficient is -0.138, and the two-tailed p-value is 0.039.


In [41]:
r, p = pearsonr(pinewood_dosenet_and_ws['radiation'], pinewood_dosenet_and_ws['Rainfall'])
print(f"Pearson's correlation coefficient is {r:.3f}, and the two-tailed p-value is {p:.3f}.")

Pearson's correlation coefficient is -0.138, and the two-tailed p-value is 0.039.
